In [ ]:
import matplotlib.pyplot as plt
from sktime.datasets import load_airline 
from sktime.utils.plotting import plot_correlations, plot_series
from sktime.transformations.series.detrend import MSTL

years = np.arange(1, 125)
index = pd.date_range(start='1900-01-01', periods=len(years), freq='YS')

# Generate the seasonal component
seasonal = pd.Series((-11 * np.sin(years / 12 * 2 * np.pi) +
            5.7 * np.sin(years / 12 * 4 * np.pi) -
            2 * np.sin(years / 12 * 6 * np.pi) +
            2.5 * np.sin(years / 12 * 8 * np.pi) +
            1.6 * np.sin(years / 12 * 10 * np.pi)), index=index)

# Generate the trend component
trend = pd.Series(40000 - (years - 1)**2, index=index)

# Generate the noise component
noise = pd.Series(np.random.normal(0, 1000, len(years)), index=index)

# Combine the components to create the time series
y = trend + seasonal + noise

# Create a Pandas Series with a DatetimeIndex
X = pd.Series(y, index=index)

X.plot()
plt.tight_layout()
plt.show()

seasonal[-25:].plot()
plt.tight_layout()
plt.show()

mstl = MSTL(return_components=True, periods=[12], windows=13) 
mstl.fit(X)

decomposition = mstl.transform(X)

decomposition.plot()
plt.tight_layout()

mstl_trafo = MSTL(return_components=True, periods=[12], windows=13) 

X = load_airline()
X.index = X.index.to_timestamp()

res = mstl_trafo.fit_transform(X)
res.plot()

X_log = np.log(X)
res = mstl_trafo.fit_transform(X_log)
res.plot()

from sktime.transformations.series.boxcox import LogTransformer
from sktime.transformations.series.detrend import MSTL
from sktime.forecasting.trend import TrendForecaster
from sktime.forecasting.compose import TransformedTargetForecaster

# Create a forecasting pipeline: MSTL for deseasonalization followed by TrendForecaster
mstl_deseason_fcst = mstl_trafo * TrendForecaster()

# Fit the forecasting pipeline to the data, forecasting horizons [1, 2, 3]
mstl_deseason_fcst.fit(X, fh=range(1,31))

# Make predictions
y_pred = mstl_deseason_fcst.predict()

from sktime.utils.plotting import plot_series

plot_series(X, y_pred)